In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
main_df = pd.read_csv("../data/raw/initial_games_data.csv")
platform_df = pd.read_csv("../data/raw/platform_game_data.csv")
categories_tags_and_genre_df = pd.read_csv("../data/raw/games.csv", index_col=False)

C:\Users\lotec\AppData\Local\Temp\ipykernel_8460\3492562591.py:3: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  steam_games_df = pd.read_csv("../data/raw/games.csv", index_col=False)


In [3]:
print(f"Main Dataset: {main_df.shape[0]} Second dataset: {platform_df.shape[0]} Third dataset: {categories_tags_and_genre_df.shape[0]}")

Main Dataset: 67571 Second dataset: 50872


In [4]:
#Picking the attributes to keep for dataset 3
categories_tags_and_genre_df = categories_tags_and_genre_df[["AppID", "Categories", "Genres", "Tags", "Price"]]
categories_tags_and_genre_df = categories_tags_and_genre_df.rename(columns={"AppID": "app_id"})

In [5]:
# Drop unnecessary index column
main_df = main_df.drop(columns=["Unnamed: 0"]) 
# Keep only the first 4 columns
main_df = main_df.iloc[:, :4] 

In [6]:
# Keeps the first occurrence of each game via link
main_df = main_df.drop_duplicates(subset='link', keep='first')

In [7]:
# Extract app_id from the link column and convert to integer
main_df["app_id"] = main_df["link"].str.extract(r'/app/(\d+)/')
main_df['app_id'] = main_df['app_id'].astype(int) 

In [8]:
# Drops the link column since we have the app_id now
main_df = main_df.drop(columns=['link'])

In [9]:
# Reorder columns to have 'app_id' first, not necessary but cleaner
cols = ['app_id'] + [c for c in main_df.columns if c != 'app_id']
main_df = main_df[cols]

In [10]:
platform_df = platform_df.drop(columns=[
    "rating", "positive_ratio", "price_original", "price_final", "discount", "steam_deck"
    ])

In [11]:
merged_df = pd.merge(main_df, platform_df, on='app_id')

In [12]:
differences = merged_df[merged_df["release"] != merged_df["date_release"]]

In [13]:
game_dupes = merged_df["game"].duplicated().sum()
title_dupes = merged_df["title"].duplicated().sum()

In [14]:
# Drops game column and date_release since we have title and release_year
merged_df = merged_df.drop(columns=["game", "date_release"])

# Merging the third dataset for categories, tags, and genre, and price
merged_df = pd.merge(
    merged_df,
    categories_tags_and_genre_df,
    on="app_id",
    how="inner" 
)

# print(merged_df.info())
print(merged_df.head(10))

matched = merged_df["Categories"].notna().sum()
print(f"Games with category info added: {matched}")


    app_id     release  peak_players                             title   win  \
0  2231450  2023-01-26          4529                       Pizza Tower  True   
1  2050650  2023-03-24        168191                   Resident Evil 4  True   
2  2324650  2023-03-31         15543  The Murder of Sonic the Hedgehog  True   
3  2263010  2023-03-28          1415                Pineapple on pizza  True   
4  1817230  2023-01-25          6132                        Hi-Fi RUSH  True   
5  2121360  2023-01-09          8672                              东北之夏  True   
6   774801  2023-04-01          8564                    Crab Champions  True   
7  2291760  2023-03-31          1227           Papa's Freezeria Deluxe  True   
8  1201270  2023-01-19           338           A Space for the Unbound  True   
9  2171690  2023-01-13           195                        Handshakes  True   

     mac  linux  price_original          Categories  \
0  False  False            0.00       Tour De Pizza   
1  False 